# Getting the Joints Data

In [1]:
import plotly.graph_objects as go
import json

In [2]:
with open("C:/Users/E278296/Downloads/2025_22_cqq2qkdt_2_2_1_5_5.baseball.action.json",'r') as file:
    data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/E278296/Downloads/2025_22_cqq2qkdt_2_2_1_5_5.baseball.action.json'

In [ ]:
samples_data = data['samples']['people']

In [ ]:
# Create dictionary to store batter hitting data
batter_joints_data = {}

# Get length of all people in the given joints file
all_people = len(samples_data)

# Loop through to get the hitter and the joint data
for i in range(all_people):

    # Ignore entry with no personId
    role_name = samples_data[i]['role']['name']
    if role_name == "Unknown":
        continue

    # Retrieve batting data
    keys = samples_data[i].keys()

    if 'personId' in keys:
        person_id_key = list(samples_data[i]['personId'].keys())[0]

    if role_name == "Batter" and  'personId' in keys:
        # Get personId and trackId
        person_id = samples_data[i]['personId'][person_id_key]
        track_id = str(samples_data[i]['trackId'])
        joints_data = samples_data[i]['joints']

        # ✅ Ensure person_id exists in the dictionary
        if person_id not in batter_joints_data:
            batter_joints_data[person_id] = {}

        # ✅ Add trackId and joints data under that person_id
        batter_joints_data[person_id][track_id] = joints_data


In [ ]:
person_id = list(batter_joints_data.keys())[0]

In [ ]:
track_id = max(list(batter_joints_data[person_id].keys()))

In [ ]:
# Not use neck, nose, eyes, ears, or fingers
{k:v for k,v in batter_joints_data[person_id][track_id][0].items() if k not in ['lEar','rEar','lEye','rEye','neck','nose','midHip','lPinky','rPinky','time']}

In [ ]:
skeleton_connections = [
    ('lAnkle','lKnee'), ('lAnkle','lBigToe'), ('lAnkle','lSmallToe'), ('lAnkle','lHeel'),
    ('rAnkle','rKnee'), ('rAnkle','rBigToe'), ('rAnkle','rSmallToe'), ('rAnkle','rHeel'),
    ('rHeel','rBigToe'), ('rHeel','rSmallToe'),
    ('lHeel','lBigToe'), ('lHeel','lSmallToe'),
    ('lKnee','lHip'), ('rKnee','rHip'),
    ('lHip','lShoulder'), ('rHip','rShoulder'),
    ('lShoulder','lElbow'), ('rShoulder','rElbow'),
    ('lElbow','lWrist'),('rElbow','rWrist'),
    ('lHip','rHip'), ('lShoulder','rShoulder'),
    ('lWrist','lThumb'), ('rWrist','rThumb'),
    ('lSmallToe','lBigToe'), ('rSmallToe','rBigToe')
    ]

# Function to create a frame
def create_frame(points):
    scatter = go.Scatter3d(
        x=[points[k][0] for k in points],
        y=[points[k][1] for k in points],
        z=[points[k][2] for k in points],
        mode='markers+text',
        marker=dict(size=5, color='blue'),
        textposition='top center',
        name='Body Parts'
    )
    lines = []
    for start, end in skeleton_connections:
        x_line = [points[start][0], points[end][0], None]
        y_line = [points[start][1], points[end][1], None]
        z_line = [points[start][2], points[end][2], None]
        lines.append(go.Scatter3d(
            x=x_line, y=y_line, z=z_line,
            mode='lines',
            line=dict(color='gray', width=2),
            showlegend=False
        ))
    return [scatter] + lines

frames = []
for time_point in batter_joints_data[person_id][track_id]:
    time = time_point['time']
    joints = {k:v for k,v in time_point.items() if k not in ['lEar','rEar','lEye','rEye','neck','nose','midHip','lPinky','rPinky','time']}
    try:
        frame_data = create_frame(joints)
        frame = go.Frame(data=frame_data, name=str(time))
        frames.append(frame)
    except KeyError:
        continue  # skip frame if any key is missing



# Define camera view
camera = dict(
    eye=dict(x=1.5, y=1.5, z=1.5),  # Diagonal top-down view
    up=dict(x=0, y=0, z=1),         # Z-axis is up
    center=dict(x=0, y=0, z=0)
)


# Initialize the figure with the first frame's data
initial_data = frames[0].data if frames else []

# Create the figure
fig = go.Figure(
    data=initial_data,
    frames=frames,
    layout=go.Layout(
        title="Skeleton Joint Animation",
        scene=dict(
            xaxis=dict(visible = False),
            yaxis=dict(visible = False),
            zaxis=dict(visible = False),
            aspectmode = 'data',
            camera = camera
        ),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(label="Play",
                         method="animate",
                         args=[None, {"frame": {"duration": 100, "redraw": True},
                                      "fromcurrent": True, "transition": {"duration": 0}}]),
                    dict(label="Pause",
                         method="animate",
                         args=[[None], {"frame": {"duration": 0, "redraw": False},
                                        "mode": "immediate",
                                        "transition": {"duration": 0}}])
                ],
                showactive=False,
                x=0.1,
                y=0,
                xanchor="right",
                yanchor="top"
            )
        ],
        sliders=[{
            "steps": [
                {
                    "method": "animate",
                    "label": frame.name,
                    "args": [[frame.name], {"frame": {"duration": 100, "redraw": True},
                                            "mode": "immediate",
                                            "transition": {"duration": 0}}]
                } for frame in frames
            ],
            "active": 0,
            "x": 0.1,
            "y": -0.1,
            "xanchor": "left",
            "yanchor": "top"
        }]
    )
)

fig.show()


# Merging the Bat Data

In [ ]:
bat_data = data['samples']['bat']

In [ ]:
bat_dict = {round(entry["time"], 2): entry for entry in bat_data}

In [ ]:
# Getting the Joints Data
import plotly.graph_objects as go
import json

samples_data = data['samples']['people']

players = data['details']['players']

teams = data['details']['teams']

# Create dictionary to store batter hitting data
batter_joints_data = {}

# Get length of all people in the given joints file
all_people = len(samples_data)

# Loop through to get the hitter and the joint data
for i in range(all_people):

    # Ignore entry with no personId
    role_name = samples_data[i]['role']['name']
    if role_name == "Unknown":
        continue

    # Retrieve batting data
    keys = samples_data[i].keys()

    if 'personId' in keys:
        person_id_key = list(samples_data[i]['personId'].keys())[0]

    if role_name == "Batter" and  'personId' in keys:
        # Get personId and trackId
        person_id = samples_data[i]['personId'][person_id_key]
        track_id = str(samples_data[i]['trackId'])
        joints_data = samples_data[i]['joints']

        # ✅ Ensure person_id exists in the dictionary
        if person_id not in batter_joints_data:
            batter_joints_data[person_id] = {}

        # ✅ Add trackId and joints data under that person_id
        batter_joints_data[person_id][track_id] = joints_data

# Not use neck, nose, eyes, ears, or fingers
{k:v for k,v in batter_joints_data[person_id][track_id][0].items() if k not in ['lEar','rEar','lEye','rEye','neck','nose','midHip','lPinky','rPinky','time']}

skeleton_connections = [
    ('lAnkle','lKnee'), ('lAnkle','lBigToe'), ('lAnkle','lSmallToe'), ('lAnkle','lHeel'),
    ('rAnkle','rKnee'), ('rAnkle','rBigToe'), ('rAnkle','rSmallToe'), ('rAnkle','rHeel'),
    ('rHeel','rBigToe'), ('rHeel','rSmallToe'),
    ('lHeel','lBigToe'), ('lHeel','lSmallToe'),
    ('lKnee','lHip'), ('rKnee','rHip'),
    ('lHip','lShoulder'), ('rHip','rShoulder'),
    ('lShoulder','lElbow'), ('rShoulder','rElbow'),
    ('lElbow','lWrist'),('rElbow','rWrist'),
    ('lHip','rHip'), ('lShoulder','rShoulder'),
    ('lWrist','lThumb'), ('rWrist','rThumb'),
    ('lSmallToe','lBigToe'), ('rSmallToe','rBigToe')
    ]

# Function to create a frame
def create_frame(points):

    joint_points = {k: v for k, v in points.items() if k not in ['bat_head', 'bat_handle'] and isinstance(v, list)}

    # Skeleton joints
    # Skeleton joints (excluding bat_head and bat_handle)
    scatter = go.Scatter3d(
        x=[joint_points[k][0] for k in joint_points if isinstance(joint_points[k], list) and k not in ['bat_head', 'bat_handle']],
        y=[joint_points[k][1] for k in joint_points if isinstance(joint_points[k], list) and k not in ['bat_head', 'bat_handle']],
        z=[joint_points[k][2] for k in joint_points if isinstance(joint_points[k], list) and k not in ['bat_head', 'bat_handle']],
        mode='markers',
        marker=dict(size=4, color='blue'),
        name='Joints'
    )

    # Skeleton connections
    lines = []
    for start, end in skeleton_connections:
        if start in points and end in points:
            x_line = [points[start][0], points[end][0], None]
            y_line = [points[start][1], points[end][1], None]
            z_line = [points[start][2], points[end][2], None]
            lines.append(go.Scatter3d(
                x=x_line, y=y_line, z=z_line,
                mode='lines',
                line=dict(color='gray', width=2),
                showlegend=False
            ))

    # Bat visuals
    bat_traces = []
    if 'bat_head' in points and 'bat_handle' in points:
        bat_traces.append(go.Scatter3d(
            x=[points['bat_head'][0]], y=[points['bat_head'][1]], z=[points['bat_head'][2]],
            mode='markers+text',
            marker=dict(size=4, color='brown'),
            name='Bat Head'
        ))
        bat_traces.append(go.Scatter3d(
            x=[points['bat_handle'][0]], y=[points['bat_handle'][1]], z=[points['bat_handle'][2]],
            mode='markers+text',
            marker=dict(size=2, color='brown'),
            name='Bat Handle'
        ))
        # Optional: line between head and handle
        bat_traces.append(go.Scatter3d(
            x=[points['bat_head'][0], points['bat_handle'][0], None],
            y=[points['bat_head'][1], points['bat_handle'][1], None],
            z=[points['bat_head'][2], points['bat_handle'][2], None],
            mode='lines',
            line=dict(color='brown', width=10),
            showlegend=False
        ))

    return [scatter] + lines + bat_traces

bat_head_trace = []
bat_handle_trace = []

person_id = max(batter_joints_data.keys())
track_id = max(batter_joints_data[person_id].keys())

for frame in batter_joints_data[person_id][track_id]:
    time = round(frame['time'], 2)

    if time in bat_dict:
        bat_entry = bat_dict[time]

        # Add bat data directly into the frame
        frame['bat_head'] = bat_entry['head']['pos']
        frame['bat_handle'] = bat_entry['handle']['pos']


#Filter the data with the joints and bat data for only entries where there is a recorded position of the bat head and the bat handle
filtered_data = [frame for frame in batter_joints_data[person_id][track_id] if 'bat_head' in frame and 'bat_handle' in frame]

# Initialize empty list to store each frame of the animation
frames = []

for time_point in filtered_data:
    time = time_point['time']

    # Accumulate bat trail
    bat_head_trace.append(time_point['bat_head'])
    bat_handle_trace.append(time_point['bat_handle'])

    joints = {k:v for k,v in time_point.items() if k not in ['lEar','rEar','lEye','rEye','neck','nose','midHip','lPinky','rPinky','time']}

    try:
        frame_data = create_frame(joints)

        # Add trail traces
        head_x, head_y, head_z = zip(*bat_head_trace)
        handle_x, handle_y, handle_z = zip(*bat_handle_trace)

        trail_traces = [
            go.Scatter3d(x=head_x, y=head_y, z=head_z,
                         mode='lines',
                         line=dict(color='red', width=2),
                         name='Bat Head Trail',
                         showlegend=False),
            go.Scatter3d(x=handle_x, y=handle_y, z=handle_z,
                         mode='lines',
                         line=dict(color='orange', width=2),
                         name='Bat Handle Trail',
                         showlegend=False)
        ]

        frame = go.Frame(data=frame_data + trail_traces, name=str(time))
        frames.append(frame)
    except KeyError:
        continue



# Define camera view
camera = dict(
    eye=dict(x=1.5, y=1.5, z=1.5),  # Diagonal top-down view
    up=dict(x=0, y=0, z=1),         # Z-axis is up
    center=dict(x=0, y=0, z=0)
)


# Initialize the figure with the first frame's data
initial_data = frames[0].data if frames else []

# Create the figure
fig = go.Figure(
    data=initial_data,
    frames=frames,
    layout=go.Layout(
        title="Skeleton Joint Animation",
        scene=dict(
            xaxis=dict(visible = False),
            yaxis=dict(visible = False),
            zaxis=dict(visible = False),
            aspectmode = 'data',
            camera = camera
        ),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(label="Play",
                         method="animate",
                         args=[None, {"frame": {"duration": 100, "redraw": True},
                                      "fromcurrent": True, "transition": {"duration": 0}}]),
                    dict(label="Pause",
                         method="animate",
                         args=[[None], {"frame": {"duration": 0, "redraw": False},
                                        "mode": "immediate",
                                        "transition": {"duration": 0}}])
                ],
                showactive=False,
                x=0.1,
                y=0,
                xanchor="right",
                yanchor="top"
            )
        ],
        sliders=[{
            "steps": [
                {
                    "method": "animate",
                    "label": frame.name,
                    "args": [[frame.name], {"frame": {"duration": 100, "redraw": True},
                                            "mode": "immediate",
                                            "transition": {"duration": 0}}]
                } for frame in frames
            ],
            "active": 0,
            "x": 0.1,
            "y": -0.1,
            "xanchor": "left",
            "yanchor": "top"
        }]
    )
)

# Select the events from the action file
events = data['events']

# Get only the event that have the type hit
hit_events = []
for event in range(len(events)):
    if events[event]['type'] == 'Hit':
        hit_events.append(events[event])

team_id = hit_events[0]['teamId']['heId']
person_id = hit_events[0]['personId'][person_id_key]

person_name = [player['fullName'] for player in players if player['id'].get(person_id_key) == person_id][0]
team_name = [team['name'] for team in teams if team['id'].get('heId') == team_id][0]

# Sweet spot deviation values
deviation_x = 0.0313
deviation_y = -0.4701

# Arbitrary position to place the text (adjust as needed)
text_x, text_y, text_z = 5, 5, 5  # You can move this to a visible corner or near the bat
axial_deviation, longitudinal_deviation = hit_events[0]['during']['bat']['sweetSpot']['deviation']

axial_deviation_text = ""

if axial_deviation == 0:
    axial_deviation_text = "Completely Full"
elif axial_deviation > 0:
    axial_deviation_text = "Towards Sky"
else:
    axial_deviation_text = "Towards Ground"

longitudinal_text = ""

if longitudinal_deviation == 0:
    longitudinal_text = "Directly on Sweet Spot"
elif longitudinal_deviation > 0:
    longitudinal_text = "Towards Head"
else:
    longitudinal_text = "Towards Handle"

# Ball position from hit_events[0]['during']['position']
contact_position = hit_events[0]['during']['bat']['contactPoint']['position']
contact_position_x, contact_position_y, contact_position_z = contact_position


# Add ball trace
fig.add_trace(go.Scatter3d(
    x=[contact_position_x],
    y=[contact_position_y],
    z=[contact_position_z],
    mode='markers+text',
    marker=dict(size=6, color='green'),
    textposition="top center",
    name="Contact Point"
))


text_x, text_y, text_z = 5, 5, 2
attack_angle = hit_events[0]['during']['bat']['impactPoint']['angle'][0]


text_x, text_y, text_z = 5, 5, 2
attack_angle = hit_events[0]['during']['bat']['impactPoint']['angle'][0]


fig.update_layout(
    annotations=[
        # Player Info
        dict(
            showarrow=False,
            text=f"Player: {person_name}<br>Team: {team_name}",
            x=0,
            y=1.05,
            xref="paper",
            yref="paper",
            align="left",
            font=dict(size=12),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1
        ),
        # Sweet Spot Deviation
        dict(
            showarrow=False,
            text=f"Sweet Spot Deviation:<br>Axial = {axial_deviation:.2f}, {axial_deviation_text}<br>Longitudinal = {longitudinal_deviation:.2f}, {longitudinal_text}",
            x=0,
            y=0.85,
            xref="paper",
            yref="paper",
            align="left",
            font=dict(size=12),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1
        ),
        # Attack Angle
        dict(
            showarrow=False,
            text=f"Attack Angle: {attack_angle:.2f}°",
            x=0,
            y=0.55,
            xref="paper",
            yref="paper",
            align="left",
            font=dict(size=12),
            bgcolor="white",
            bordercolor="black",
            borderwidth=1
        )
    ]
)

fig.show()